In [1]:
!pip install pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ca66e075fe94e3c7c6986110fd3030f93831682739f1da43c6cf2a9f92a45613
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Khi ta tạo một SparkContext, Tham số Main gán một tên do ta chọn cho các công việc Spark được thực thi. Tham số master chỉ ra rằng các công việc Spark sẽ chạy ở chế độ cục bộ, sử dụng hai luồng. Điều này có nghĩa là các công việc Spark của chúng ta thực sự không chạy trên một cụm, mà thay vào đó chạy trong một quy trình đơn lẻ trên máy cục bộ. Ta lập trình các công việc Spark theo cùng một cách dù chúng chạy ở chế độ cục bộ hay trên một cụm - sự khác biệt chính giữa chế độ cục bộ và cụm, tất nhiên, là hiệu suất.


In [2]:
import os
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("Main")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Trong đoạn mã tiếp theo, parallelize() và map() là các phép biến đổi của Spark, và reduce() là action trong Spark.Chạy code bên dưới, một job Spark sẽ được tạo và thực thi, và nó sẽ in ra một estimation của e khi hoàn thành.

In [9]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


Nó bắt đầu bằng cách sử dụng SparkContext (sc), cho Python biết rằng chúng ta đang sử dụng các chức năng của Spark được triển khai trong lớp SparkContext. Sau đó, range (0, n) được parallize, nghĩa là, nó được "chuyển đổi" thành một RDD, một tập hợp parallize các đối tượng trong Spark mà dữ liệu của nó được phân vùng trên nhiều cụm (trong trường hợp này là một nút đơn lẻ).

Tiếp theo, RDD được chuyển đổi thêm (tạo ra một RDD khác) bằng cách sử dụng thao tác map, trong ví dụ tính $e$, chuyển đổi các số nguyên thành các hạng tử riêng lẻ của dãy số đã cho. Điều đáng nói là RDD chưa thực hiện bất kỳ chuyển đổi nào cho đến thời điểm hiện tại. Một chương trình Spark duy trì thứ tự thực thi dưới dạng DAG (Directed Acyclic Graph) trong đó mỗi đỉnh biểu thị một RDD và mỗi cạnh trước đó biểu thị thao tác đã tạo ra RDD đó. Nói cách khác, mỗi RDD duy trì một con trỏ đến RDD cha của nó duy trì phép biến đổi đã tạo ra nó. Điều này được gọi là phả hệ (lineage).

Quay trở lại với code estimation $e$, RDD được ánh xạ sau đó được giảm xuống một giá trị duy nhất bằng cách tính tổng chạy của từng hạng tử liên tiếp, tổng hợp tất cả các hạng tử của dãy số đã cho. reduce() này là action nộp DAG cho bộ lập lịch DAG trong Spark và dẫn đến việc tính toán các RDD đã chỉ định. Cuối cùng, chúng ta lấy nghịch đảo của dãy số sử dụng Python để ước tính giá trị của e.

In [6]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

In [4]:
file = "Shakespeare.txt"

In [7]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens(sc, file):

    text_file = sc.textFile(file)

    # Works and is efficient because of combinining although we don't want sum for now.
    text_take = text_file.flatMap(lambda x: simple_tokenize(x)).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).count()
    return text_take

In [8]:
count_distinct_tokens(sc, file)

25975